In [126]:
#Suzan Iloglu, May 21,2020
import gurobipy as gp
from gurobipy import GRB
from itertools import product
import pandas as pd
import numpy as np


In [127]:
from IPython.display import Image

![Contact Tracing Allocation](image1.png)

## Contact Tracing Coverage Model

### Objective and Prerequisites

In this model, we solve capacitated coverage model: how to create contact tracer centers to provide coverage for the people who need to be traced with the goal of maximizing the cumulative weighted total number of people traced. We implement this model in the Gurobi Python interface and compute optimal solution

### Motivation


### Problem Description


### Model Formulation 
 
#### Sets
$I $ set of contact tracing demand points (census tracts)<br>
$J$  set of contact tracer center locations (counties)<br>
$S$  set of states<br>
$N_i$ set of contact tracer center locations that can serve demand point $i \in I$<br>
$M_j$ set of contact tracer center locations that belongs to state $s \in S$<br>

#### Parameters
$w_{i}$  weight assigned to demand point $i \in I$ <br>
$d_{i}$  total number of contact tracer needed at demand point $i \in I$<br>
$z_{j}$  weight assigned to contact tracer location  $j \in J$<br>
$c_{s}$  total number of contact tracer capacity at each state $s \in S$<br>


The decision variables for the model are as follows: <br>
 $y_{j} = $the number of contact tracer at center $j \in J$. <br>
 $x_{ij} =$ the number of community health care workers assigned to demand point $i \in I$ by contact tracing center $j \in J$, and 0 otherwise. <br>



The integer programming formulation of our model is as follows.
<br>
$$
\begin{align}
    \max & \sum_{j \in J} \sum_{i \in I} w_{i}x_{ij} + \sum_{j \in J} z_{j}y_{j} && \\
    \text{s.t. } &\sum_{j \in N_{i}}  x_{ij} \leq  d_i &\text{ for }& i \in I  \\
    & \sum_{i \in I: j \in N_{i}} x_{ij} \leq y_{j} \ &\text{ for }& j \in J \\
    & \sum_{j \in M_s}  y_{j} \leq c_s &\text{ for }& s \in S  \\
    & y_{j} \leq d_j &\text{ for }& j \in J  \\
    & x_{ij}  \text{ integer }&\text{  for }& i\in I, j \in J \\
    & y_{j}  \text{ integer }&  \text{ for } &j \in J 
\end{align}
$$

In [128]:
df_1 = pd.read_csv('County_based_demand.csv')
df_1.head(10)

,FIPS,County_Name,State,Population,14-Day_Case_Load,County_Baseline_CT_Need,Total_COVID_Need,Final_Need
0,1001,Autauga,AL,55869,45,9,23,23
1,1003,Baldwin,AL,223234,56,34,27,34
2,1005,Barbour,AL,24686,26,4,12,12
3,1007,Bibb,AL,22394,4,4,3,4
4,1009,Blount,AL,57826,11,9,5,9
5,1011,Bullock,AL,10101,14,2,8,8
6,1013,Butler,AL,19448,162,3,72,72
7,1015,Calhoun,AL,113605,34,18,15,18
8,1017,Chambers,AL,33254,30,5,14,14
9,1019,Cherokee,AL,26196,10,4,6,6


In [129]:
TextFileReader = pd.read_csv('svi_2018_tracts_state_ranked.csv',chunksize=5000)
df_svi_x = []
for df in TextFileReader:
    df_svi_x.append(df)

df_svi = pd.concat(df_svi_x, sort=False)

df_svi.head(5)
#df_svi.shape

#print ('merged',df.count)
#print ('before merged', df_1.count)

,Unique_Number,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,AREA_SQMI,E_TOTPOP,M_TOTPOP,RPL_ThemesStates
0,70248,45,SOUTH CAROLINA,SC,45001,Abbeville,45001950200,73.878384,4128,293,0.572878
1,61884,45,SOUTH CAROLINA,SC,45001,Abbeville,45001950100,48.480203,2855,295,0.632841
2,61885,45,SOUTH CAROLINA,SC,45001,Abbeville,45001950300,96.429397,3333,359,0.453875
3,36715,45,SOUTH CAROLINA,SC,45001,Abbeville,45001950600,84.876057,3122,392,0.770295
4,36714,45,SOUTH CAROLINA,SC,45001,Abbeville,45001950400,77.552106,6197,417,0.880074


In [130]:
county_svi = pd.read_csv('SVI_county_based_per_State.csv', encoding='latin-1')
county_svi.head(5)

,Unique_Number,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,RPL_ThemesStates
0,1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,...,0,0,3875,508,7.1,0.9,37301,0.150256,2.052740,0.104478
1,1315,1,ALABAMA,AL,Baldwin,1003,"Baldwin County, Alabama",1589.793007,208107,0,...,1,1,20864,1646,10.2,0.8,195677,0.409925,4.277934,0.029851
2,3076,1,ALABAMA,AL,Barbour,1005,"Barbour County, Alabama",885.001636,25782,0,...,2,8,2558,363,11.2,1.6,25052,0.223255,2.567149,1.000000
3,2130,1,ALABAMA,AL,Bibb,1007,"Bibb County, Alabama",622.461089,22527,0,...,2,2,1619,396,7.9,1.9,17696,0.156525,1.886955,0.298507
4,2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,...,0,0,6303,732,11.0,1.3,40036,0.164403,2.392326,0.074627


In [131]:
df_2 = pd.merge(left = df_1, right = county_svi, how = 'right', right_on = 'FIPS', left_on = 'FIPS' )
df_2.head(10)

,FIPS,County_Name,State,Population,14-Day_Case_Load,County_Baseline_CT_Need,Total_COVID_Need,Final_Need,Unique_Number,ST,...,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_STAr,Shape_STLe,RPL_ThemesStates
0,1001,Autauga,AL,55869,45,9,23,23,1,1,...,0,0,3875,508,7.1,0.9,37301,0.150256,2.052740,0.104478
1,1003,Baldwin,AL,223234,56,34,27,34,1315,1,...,1,1,20864,1646,10.2,0.8,195677,0.409925,4.277934,0.029851
2,1005,Barbour,AL,24686,26,4,12,12,3076,1,...,2,8,2558,363,11.2,1.6,25052,0.223255,2.567149,1.000000
3,1007,Bibb,AL,22394,4,4,3,4,2130,1,...,2,2,1619,396,7.9,1.9,17696,0.156525,1.886955,0.298507
4,1009,Blount,AL,57826,11,9,5,9,2,1,...,0,0,6303,732,11.0,1.3,40036,0.164403,2.392326,0.074627
5,1011,Bullock,AL,10101,14,2,8,8,3028,1,...,2,7,1076,335,10.8,3.4,8399,0.154693,2.043525,0.820896
6,1013,Butler,AL,19448,162,3,72,72,3,1,...,0,0,2005,340,10.2,1.7,17280,0.191747,1.818327,0.761194
7,1015,Calhoun,AL,113605,34,18,15,18,4,1,...,0,0,10686,796,9.4,0.7,117894,0.154336,2.194795,0.686567
8,1017,Chambers,AL,33254,30,5,14,14,5,1,...,0,0,3613,476,10.8,1.4,27176,0.150588,1.714193,0.522388
9,1019,Cherokee,AL,26196,10,4,6,6,1316,1,...,1,1,2133,432,8.3,1.7,19502,0.151939,1.886092,0.134328


In [132]:

df = pd.merge(left = df_2, right = df_svi, how = 'right', right_on = 'STCNTY', left_on = 'FIPS' )

#df.to_dict('list')
df['RPL_ThemesStates_y'] = df['RPL_ThemesStates_y'].fillna(0)

#df.type


df.head(10)
#df.shape

,FIPS_x,County_Name,State,Population,14-Day_Case_Load,County_Baseline_CT_Need,Total_COVID_Need,Final_Need,Unique_Number_x,ST_x,...,ST_y,STATE_y,ST_ABBR_y,STCNTY,COUNTY_y,FIPS_y,AREA_SQMI_y,E_TOTPOP_y,M_TOTPOP_y,RPL_ThemesStates_y
0,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020200,1.284051,2028,192,0.617547
1,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001021000,149.369459,2550,239,0.580068
2,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020100,3.790677,1923,253,0.155026
3,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020801,47.981925,2826,324,0.121806
4,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001021100,184.615348,2945,335,0.706985
5,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020400,2.464982,3831,337,0.097104
6,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020600,3.104879,3705,342,0.465928
7,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020300,2.065365,3476,433,0.547700
8,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020900,113.034040,6401,532,0.179727
9,1001,Autauga,AL,55869,45,9,23,23,1,1,...,1,ALABAMA,AL,1001,Autauga,1001020700,8.652778,4029,545,0.886712


In [133]:
#df.FIPS_y.apply(str)
demand_point = df.FIPS_y.tolist()
population_census = df.E_TOTPOP_y.tolist()
demand_amount = df.Final_Need.tolist()
State_county = df.State.tolist()
CT_location = df.FIPS_x.tolist()
weight_list = df.RPL_ThemesStates_y.tolist()
county_weight_list = df.RPL_THEMES.tolist()

In [134]:
population_county = df.groupby(['FIPS_x'])['E_TOTPOP_y'].sum().to_dict()
demand_per_county = dict(zip(df.FIPS_x, df.Final_Need))
capacity_county = dict(zip(df.FIPS_x, df.County_Baseline_CT_Need))
#print (capacity_county)
#print (demand_per_county)
#print (population_county)
#print (CT_location)
#print (demand_point)
FIPS_County_Name_dict = dict(zip(df.FIPS_y, df.COUNTY_y))
county_of_states = dict(zip(df.FIPS_x, df.State))
county_name = dict(zip(df.FIPS_x, df.COUNTY_y))
census_county = dict(zip(df.FIPS_y, df.FIPS_x))
county_weight = dict(zip(df.FIPS_x, df.RPL_THEMES))
#print (county_of_states)
#print(county_name)

In [135]:
coverage_loc_set = {}
demand = {}
weight = {}
population = {}
county_pop = {}
count = 0
county_demand = {}


for d in demand_point:
    county_demand[census_county[d]] = demand_amount[count]
    county_pop[d] = population_county[census_county[d]]
    population[d] = population_census[count]
    coverage_loc_set[d] = CT_location[count]
    if county_pop[d] >= 1e-6:
        demand[d] = (demand_amount[count]*population[d])/county_pop[d]
    else:
        demand[d] = 0
    #print (d, demand[d], np.floor(demand[d]), np.ceil(demand[d]), demand_amount[count], population[d], county_pop[d])
    weight[d] = weight_list[count]  
    count += 1     
    

    #print (d, demand[d],demand_amount[count])
#print (sum (demand_amount)) 
#print (sum(demand))
#print (county_demand)

In [136]:
#others = list(df.columns)
#others.remove('FIPS')

In [137]:
#demand_point, demand = gp.multidict({demand_p:demand} for demand in demand_county for demand_p in demand_point_x)

new_df = df.drop_duplicates(subset=['FIPS_x'])
capacity = new_df.groupby(['State'])['County_Baseline_CT_Need'].sum().to_dict()

demand_per_State = new_df.groupby(['State'])['Final_Need'].sum().to_dict()
population_per_state =  new_df.groupby(['State'])['Population'].sum().to_dict()
#print (demand_per_State)
not_enough = 0
USA_capacity = 0
for i in capacity:
    USA_capacity += capacity[i]
    if capacity[i] != demand_per_State[i]:
        not_enough += 1
        #print (i, capacity[i], demand_per_State[i], (population_per_state[i]/100000)*15)
#print (not_enough)
#print (USA_capacity)

In [138]:
#Parameters
Budget = 300000
#print (Budget)
#demand_point, weight, demand, coverage_loc_set = gp.multidict({0:[8,5,{0}], 1:[7,1,{0}], 2:[5,10,{0}] ,3:[6,1,{0}] ,4:[10,3,{0}] ,5:[5,2,{0}], 6:[7,6,{0}], 7:[5,4,{0}]})
#CT_location, capacity = gp.multidict({0: 10})
#for i in demand_point:
 #   print (i,coverage_loc_set[i])
location = list(dict.fromkeys(CT_location))
pro = [(i,coverage_loc_set[i]) for i in demand_point]
#print (location)
#print (pro)
pro_c_s = [(i,county_of_states[i]) for i in location]
#print (pro_c_s)

In [139]:

cartesian_prod = gp.tuplelist(pro)
cartesian_pro_county_state = gp.tuplelist(pro_c_s)
#print (cartesian_prod)


#for (i,j) in cartesian_prod:
 #   if j not in coverage_loc_set[i]:
  #      cartesian_prod.remove((i,j))
       # print (i,j)
   # if j in coverage_loc_set[i]:
    #    print (i,j)
#print (cartesian_pro_county_state)

In [140]:
#MIP model formulation
m = gp.Model("Contact_Tracing_Coverage")

In [141]:

#Add variable for each contact tracer center
y = m.addVars(location, vtype = GRB.INTEGER, name = "y")


#Add variable for each demand point (census tract/block)
x = m.addVars(cartesian_prod, vtype = GRB.CONTINUOUS, name = "x")
m.update()


In [142]:
#for j in CT_location:
    #print (j)
#for i in demand_point:
    #print (i)
    #for j in CT_location:
        #print (j)
        #if j in coverage_loc_set[i]:
            #print (j, 'can cover', i)

In [143]:
#Coverage constraint

m.addConstrs(gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select(i,'*')) <= demand[i] for i in demand_point)

{1001020200: <gurobi.Constr *Awaiting Model Update*>,
 1001021000: <gurobi.Constr *Awaiting Model Update*>,
 1001020100: <gurobi.Constr *Awaiting Model Update*>,
 1001020801: <gurobi.Constr *Awaiting Model Update*>,
 1001021100: <gurobi.Constr *Awaiting Model Update*>,
 1001020400: <gurobi.Constr *Awaiting Model Update*>,
 1001020600: <gurobi.Constr *Awaiting Model Update*>,
 1001020300: <gurobi.Constr *Awaiting Model Update*>,
 1001020900: <gurobi.Constr *Awaiting Model Update*>,
 1001020700: <gurobi.Constr *Awaiting Model Update*>,
 1001020500: <gurobi.Constr *Awaiting Model Update*>,
 1001020802: <gurobi.Constr *Awaiting Model Update*>,
 1003011408: <gurobi.Constr *Awaiting Model Update*>,
 1003011405: <gurobi.Constr *Awaiting Model Update*>,
 1003011406: <gurobi.Constr *Awaiting Model Update*>,
 1003011102: <gurobi.Constr *Awaiting Model Update*>,
 1003011300: <gurobi.Constr *Awaiting Model Update*>,
 1003011202: <gurobi.Constr *Awaiting Model Update*>,
 1003010903: <gurobi.Constr 

In [144]:
#m.addConstrs(z[i] <= gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select(i,'*')) for i in demand_point)

In [145]:
#Each demand point can be covered at most one location
m.addConstrs(gp.quicksum(x[i,j] for (i,j) in cartesian_prod.select('*',j)) <=   y[j] for j in location)

{1001: <gurobi.Constr *Awaiting Model Update*>,
 1003: <gurobi.Constr *Awaiting Model Update*>,
 1005: <gurobi.Constr *Awaiting Model Update*>,
 1007: <gurobi.Constr *Awaiting Model Update*>,
 1009: <gurobi.Constr *Awaiting Model Update*>,
 1011: <gurobi.Constr *Awaiting Model Update*>,
 1013: <gurobi.Constr *Awaiting Model Update*>,
 1015: <gurobi.Constr *Awaiting Model Update*>,
 1017: <gurobi.Constr *Awaiting Model Update*>,
 1019: <gurobi.Constr *Awaiting Model Update*>,
 1021: <gurobi.Constr *Awaiting Model Update*>,
 1023: <gurobi.Constr *Awaiting Model Update*>,
 1025: <gurobi.Constr *Awaiting Model Update*>,
 1027: <gurobi.Constr *Awaiting Model Update*>,
 1029: <gurobi.Constr *Awaiting Model Update*>,
 1031: <gurobi.Constr *Awaiting Model Update*>,
 1033: <gurobi.Constr *Awaiting Model Update*>,
 1035: <gurobi.Constr *Awaiting Model Update*>,
 1037: <gurobi.Constr *Awaiting Model Update*>,
 1039: <gurobi.Constr *Awaiting Model Update*>,
 1041: <gurobi.Constr *Awaiting Model Up

In [146]:
#for i in demand_point :
 #   print (i,demand[i])
#total_cap = 0
#for j in CT_location:
 #   total_cap += capacity[j]
  #  print (j, capacity[j]/demand_per_State[j])

In [147]:
#for i in demand_point:
 #   print (i, FIPS_County_Name_dict[i], demand[i]/population[i], 'demand', demand[i], population[i])

In [148]:
#for j in CT_location:
    #print (j, capacity[j]/demand_per_State[j], demand_per_State[j], capacity[j] )

In [149]:
#m.addConstrs(gp.quicksum((x[i,j]/demand[i]) for (i,j) in cartesian_prod.select(i,'*') ) >= -0.003+ min( capacity[j]/demand_per_State[j] for j in CT_location) for i in demand_point)

In [150]:
#for (j,s) in cartesian_pro_county_state:
 #   print (j,s, capacity_State[s], y[j])


In [151]:

for s in capacity:
    #print (s,capacity[s])
    m.addConstr(gp.quicksum(y[j] for j in location if (j,s) in cartesian_pro_county_state) <= ((70/15)*capacity[s]))

In [152]:
m.addConstrs(y[j] <=   demand_per_county[j] for j in location)

{1001: <gurobi.Constr *Awaiting Model Update*>,
 1003: <gurobi.Constr *Awaiting Model Update*>,
 1005: <gurobi.Constr *Awaiting Model Update*>,
 1007: <gurobi.Constr *Awaiting Model Update*>,
 1009: <gurobi.Constr *Awaiting Model Update*>,
 1011: <gurobi.Constr *Awaiting Model Update*>,
 1013: <gurobi.Constr *Awaiting Model Update*>,
 1015: <gurobi.Constr *Awaiting Model Update*>,
 1017: <gurobi.Constr *Awaiting Model Update*>,
 1019: <gurobi.Constr *Awaiting Model Update*>,
 1021: <gurobi.Constr *Awaiting Model Update*>,
 1023: <gurobi.Constr *Awaiting Model Update*>,
 1025: <gurobi.Constr *Awaiting Model Update*>,
 1027: <gurobi.Constr *Awaiting Model Update*>,
 1029: <gurobi.Constr *Awaiting Model Update*>,
 1031: <gurobi.Constr *Awaiting Model Update*>,
 1033: <gurobi.Constr *Awaiting Model Update*>,
 1035: <gurobi.Constr *Awaiting Model Update*>,
 1037: <gurobi.Constr *Awaiting Model Update*>,
 1039: <gurobi.Constr *Awaiting Model Update*>,
 1041: <gurobi.Constr *Awaiting Model Up

In [153]:
#scalar = 0.00005
#for i in demand_point :
#    print (i, FIPS_County_Name_dict[i], weight[i], demand[i]/sum(demand_per_State[j] for (i,j) in cartesian_prod.select(i,'*')))

In [154]:
hot_stop_weight = {} #demand divided by pop if no pop then it is 0
total = 0
for j in location:
    
    if population_county[j] <1e-6:
        hot_stop_weight[j] = 0
    else:
        hot_stop_weight[j] = demand_per_county[j]/population_county[j]
        total += hot_stop_weight[j]
        #print(j, county_demand[j], demand_per_county[j], population_county[j])

max_hot = max(hot_stop_weight[j] for j in location)
min_hot = min(hot_stop_weight[j] for j in location)
divide_multip = 1/(max_hot-min_hot)
print (total) 
print (max(hot_stop_weight[j] for j in location))
print (min(hot_stop_weight[j] for j in location))

1.442969006564771
0.045649221769560225
0.0001432870038687491


In [155]:
m.setObjective(gp.quicksum(weight[i]*population[i]*x[i,j] for (i,j) in cartesian_prod ) + gp.quicksum((county_weight[j]*hot_stop_weight[j]*100000)*y[j] for j in location), GRB.MAXIMIZE)

In [156]:
m.update()
m.write("CT_coverage_model.lp")
#m.computeIIS()
#m.write("model.ilp")

In [157]:
m.update()
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 79172 rows, 75979 columns and 155100 nonzeros
Model fingerprint: 0x3b921f81
Variable types: 72837 continuous, 3142 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-04, 3e+04]
Found heuristic solution: objective 3.894421e+08
Presolve removed 79110 rows and 71100 columns
Presolve time: 0.16s
Presolved: 62 rows, 4879 columns, 4940 nonzeros
Found heuristic solution: objective 4.261937e+08
Variable types: 4817 continuous, 62 integer (1 binary)

Root relaxation: objective 4.324246e+08, 71 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.3242e+08    0   59 4.2619e+08 4.3242e+08  1.46%     -    0s
H    0     0                    4.324217e+08 4.3242e+08  0.00%  

In [158]:

#print (m.display())

In [159]:
covered_or_not = []
demand_c = []
location_c = []
percentage = []
percentage_d = {}
county_name_t = []
state_name = []

for i in demand_point:
    if (sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*')) < 1e-6 ):
        #print (f"\n Demand point {FIPS_County_Name_dict[i]} with FIPS {i} with total demand {demand[i]} and SVI {weight[i]} is NOT covered by contact tracer location {coverage_loc_set [i]}, {county_of_states[FIPS_County_Name_dict[i]]}.")
        covered_or_not.append(0)
        demand_c.append(str(i).zfill(11))
        county_name_t.append(FIPS_County_Name_dict[i])
        state_name.append(county_of_states[census_county[i]])
        if demand[i] > 1e-6:
            percentage.append(100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i])
            percentage_d [i] = 100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i]
            
        else:
            percentage.append(0)
            percentage_d [i] = 0
            
        #print (i,0)
    elif (sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*')) > 1e-6 ):
        #print (f"\n Demand point {FIPS_County_Name_dict[i]} with FIPS {i} with total demand {demand[i]} and SVI {weight[i]}: covered demand {sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))} by contact tracer location {coverage_loc_set [i]}, {county_of_states[FIPS_County_Name_dict[i]]}.")
        county_name_t.append(FIPS_County_Name_dict[i])
        state_name.append(county_of_states[census_county[i]])        
        covered_or_not.append(1)
        demand_c.append(str(i).zfill(11))
        if demand[i] > 1e-6:
            percentage.append(100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i])
            percentage_d [i] = 100*sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select(i,'*'))/demand[i]
        else:
            percentage.append(0)
            percentage_d [i] = 0
        #print (i,1)



demand_covered_per_state = {}
percentage_covered_per_state = {}
capacity_enough = {}
county_flow = []
county_percentage = []
county_name_c = []
state_name_c = []
demand_of_county = []
hot_spot = []
for j in location:
    location_c.append(str(j).zfill(5))
    hot_spot.append(100*hot_stop_weight[j])
    
    county_flow.append(y[j].x)
    demand_of_county.append(demand_per_county[j])
    county_percentage.append((y[j].x/demand_per_county[j])*100)
    county_name_c.append(county_name[j])
    state_name_c.append(county_of_states[j])
    #print (str(j).zfill(5), y[j].x, demand_per_county[j], capacity_county[j], county_name[j], county_of_states[j])
    
for j in capacity:
    if capacity[j] != demand_per_State[j]:
        capacity_enough[j] = 0
    else :
        capacity_enough[j] = 1
        
#for j in CT_location:
#    demand_covered_per_state[j] = sum(abs(x[i,j].x) for (i,j) in cartesian_prod.select('*',j))
#    percentage_covered_per_state[j] = (demand_covered_per_state[j]/demand_per_State[j])*100

#    print (f"\n State {j} can cover  {round(percentage_covered_per_state[j],2)} % of demand and capacity situation is {capacity_enough[j]}")
print (location_c)   

['01001', '01003', '01005', '01007', '01009', '01011', '01013', '01015', '01017', '01019', '01021', '01023', '01025', '01027', '01029', '01031', '01033', '01035', '01037', '01039', '01041', '01043', '01045', '01047', '01049', '01051', '01053', '01055', '01057', '01059', '01061', '01063', '01065', '01067', '01069', '01071', '01073', '01075', '01077', '01079', '01081', '01083', '01085', '01087', '01089', '01091', '01093', '01095', '01097', '01099', '01101', '01103', '01105', '01107', '01109', '01111', '01113', '01115', '01117', '01119', '01121', '01123', '01125', '01127', '01129', '01131', '01133', '02013', '02016', '02020', '02050', '02060', '02068', '02070', '02090', '02100', '02105', '02110', '02122', '02130', '02150', '02158', '02164', '02170', '02180', '02185', '02188', '02195', '02198', '02220', '02230', '02240', '02261', '02275', '02282', '02290', '04001', '04003', '04005', '04007', '04009', '04011', '04012', '04013', '04015', '04017', '04019', '04021', '04023', '04025', '04027', 

In [160]:
#for (i,j) in cartesian_prod:
 #   if (abs(x[i,j].x) > 1e-6 ):
  #      print (f"\n Demand point {i} with {x[i,j].x} of total demand {demand[i]} with percentage {percentage_d[i]} is covered by contact tracer center {j}, {county_of_states[j]}.")

In [161]:
import csv

writefile = '../Contact_Tracer_partial_demand_and_coverage_USA_county_census.csv'
fieldnames = ['Census Tract FIPS', 'demand_covered','percentage', 'county_name', 'state_name']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    for row in zip(demand_c, covered_or_not, percentage, county_name_t, state_name):
        #print (row)
        writer.writerow(row)

In [162]:
#import geopandas as gpd
#from census import Census
#import matplotlib.pyplot as plt
#import contextily as ctx
#pd.options.display.max_columns =200
#c = Census("d95e144b39e17f929287714b0b8ba9768cecdc9f")

In [163]:
writefile = '../Contact_Tracer_partial_demand_and_coverage_USA_county_COVERAGE_census.csv'
fieldnames = ['County FIPS', 'demand_covered','total_demand_of_county','percentage', 'hot_spot', 'county_name', 'state_name']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    for row in zip(location_c, county_flow, demand_of_county, county_percentage, hot_spot, county_name_c, state_name_c):
        print (row)
        writer.writerow(row)

('01001', 23.0, 23, 100.0, 0.04166666666666667, 'Autauga', 'AL')
('01003', 34.0, 34, 100.0, 0.01633774933087306, 'Baldwin', 'AL')
('01005', 12.0, 12, 100.0, 0.046544100535257156, 'Barbour', 'AL')
('01007', 4.0, 4, 100.0, 0.017756470013761263, 'Bibb', 'AL')
('01009', 9.0, 9, 100.0, 0.0156128024980484, 'Blount', 'AL')
('01011', 8.0, 8, 100.0, 0.07727975270479134, 'Bullock', 'AL')
('01013', 72.0, 72, 100.0, 0.3595505617977528, 'Butler', 'AL')
('01015', 18.0, 18, 100.0, 0.01563884689568889, 'Calhoun', 'AL')
('01017', 14.0, 14, 100.0, 0.04138828120380772, 'Chambers', 'AL')
('01019', 6.0, 6, 100.0, 0.02320813832050439, 'Cherokee', 'AL')
('01021', 8.0, 8, 100.0, 0.01821078989301161, 'Chilton', 'AL')
('01023', 13.0, 13, 100.0, 0.09942638623326959, 'Choctaw', 'AL')
('01025', 17.0, 17, 100.0, 0.0697092713330873, 'Clarke', 'AL')
('01027', 4.0, 4, 100.0, 0.02989983555090447, 'Clay', 'AL')
('01029', 3.0, 3, 100.0, 0.020083009773731425, 'Cleburne', 'AL')
('01031', 21.0, 21, 100.0, 0.0409452503509592

('12123', 4.0, 4, 100.0, 0.01810118562765861, 'Taylor', 'FL')
('12125', 3.0, 3, 100.0, 0.019686331124089507, 'Union', 'FL')
('12127', 83.0, 83, 100.0, 0.01573060113639379, 'Volusia', 'FL')
('12129', 6.0, 6, 100.0, 0.018822348401668916, 'Wakulla', 'FL')
('12131', 15.0, 15, 100.0, 0.022776276230678127, 'Walton', 'FL')
('12133', 4.0, 4, 100.0, 0.01628266710087112, 'Washington', 'FL')
('13001', 14.0, 14, 100.0, 0.07586431126043133, 'Appling', 'GA')
('13003', 6.0, 6, 100.0, 0.07259528130671505, 'Atkinson', 'GA')
('13005', 11.0, 11, 100.0, 0.09796936230851443, 'Bacon', 'GA')
('13007', 6.0, 6, 100.0, 0.18814675446848542, 'Baker', 'GA')
('13009', 32.0, 32, 100.0, 0.07066201475069558, 'Baldwin', 'GA')
('13011', 7.0, 7, 100.0, 0.037817396002161, 'Banks', 'GA')
('13013', 32.0, 32, 100.0, 0.04161951955467114, 'Barrow', 'GA')
('13015', 29.0, 29, 100.0, 0.02798687512063308, 'Bartow', 'GA')
('13017', 8.0, 8, 100.0, 0.04663635303719249, 'Ben Hill', 'GA')
('13019', 3.0, 3, 100.0, 0.01576872536136662, '

('33007', 5.0, 5, 100.0, 0.01560646732005743, 'Coos', 'NH')
('33009', 14.0, 14, 100.0, 0.015588290966585386, 'Grafton', 'NH')
('33011', 284.0, 284, 100.0, 0.06908513283076331, 'Hillsborough', 'NH')
('33013', 45.0, 45, 100.0, 0.030110001873511226, 'Merrimack', 'NH')
('33015', 142.0, 142, 100.0, 0.04653769389340246, 'Rockingham', 'NH')
('33017', 30.0, 30, 100.0, 0.023394184205806436, 'Strafford', 'NH')
('33019', 7.0, 7, 100.0, 0.016231884057971015, 'Sullivan', 'NH')
('34001', 202.0, 330, 61.212121212121204, 0.12288717839866836, 'Atlantic', 'NJ')
('34003', 339.0, 782, 43.350383631713555, 0.08408611192055046, 'Bergen', 'NJ')
('34005', 103.0, 531, 19.397363465160076, 0.1189604070193361, 'Burlington', 'NJ')
('34007', 377.0, 785, 48.02547770700637, 0.15472035035782777, 'Camden', 'NJ')
('34009', 17.0, 81, 20.98765432098765, 0.08644149191611973, 'Cape May', 'NJ')
('34011', 281.0, 349, 80.51575931232091, 0.22750977835723601, 'Cumberland', 'NJ')
('34013', 542.0, 1095, 49.49771689497717, 0.1379866

('40117', 3.0, 3, 100.0, 0.018261504747991236, 'Pawnee', 'OK')
('40119', 13.0, 13, 100.0, 0.015948571989400334, 'Payne', 'OK')
('40121', 7.0, 7, 100.0, 0.015772159884637916, 'Pittsburg', 'OK')
('40123', 6.0, 6, 100.0, 0.01564210855623338, 'Pontotoc', 'OK')
('40125', 11.0, 11, 100.0, 0.015277777777777777, 'Pottawatomie', 'OK')
('40127', 2.0, 2, 100.0, 0.017987229067362172, 'Pushmataha', 'OK')
('40129', 1.0, 1, 100.0, 0.026968716289104636, 'Roger Mills', 'OK')
('40131', 14.0, 14, 100.0, 0.015416125267029313, 'Rogers', 'OK')
('40133', 6.0, 6, 100.0, 0.023932033026205578, 'Seminole', 'OK')
('40135', 7.0, 7, 100.0, 0.016924974008075633, 'Sequoyah', 'OK')
('40137', 7.0, 7, 100.0, 0.01591523997908283, 'Stephens', 'OK')
('40139', 144.0, 144, 100.0, 0.6817859002888121, 'Texas', 'OK')
('40141', 8.0, 8, 100.0, 0.10645375914836992, 'Tillman', 'OK')
('40143', 98.0, 98, 100.0, 0.015246250278088493, 'Tulsa', 'OK')
('40145', 13.0, 13, 100.0, 0.01669877970456005, 'Wagoner', 'OK')
('40147', 28.0, 28, 10

In [164]:
#shape_file = gpd.read_file('SVI2018_US_COUNTY.zip')